In [3]:
!pip install torch scikit-learn pandas numpy matplotlib joblib

In [4]:
from google.colab import files
uploaded = files.upload()

Saving news_model.pth to news_model.pth
Saving test.csv to test.csv
Saving tfidf.pkl to tfidf.pkl


In [5]:
import pandas as pd

df = pd.read_csv("test.csv")
df.head()

,text,label
0,"[코인 시황] BTC, 8만6000달러대 정체…AI 테마 약화 속 상승 동력 부족 ...",1
1,기준금리 4연속 동결 유력…가계대출 금리는 열달만에 반등 이자 부담을 가중시키는 한...,1
2,李 공약 ‘서민금융안정기금’ 속도…은행 출연금 상시화 추진 26일 금융권과 <b>정...,1
3,"안타 1개에 백미밥 1세트 적립…농협·NC다이노스, 1000세트 기부 보육원·<b>...",0
4,"제천시, 고향맛집 한종석 대표 이웃돕기 성금 기탁 <b>It</b> is assum...",1


In [6]:
import joblib

vectorizer = joblib.load("tfidf.pkl")

X_test = vectorizer.transform(df["text"]).toarray()
y_test = df["label"].values

In [7]:
import torch
import torch.nn as nn

class NewsClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

model = NewsClassifier(X_test.shape[1])
model.load_state_dict(torch.load("news_model.pth"))
model.eval()


NewsClassifier(
  (fc1): Linear(in_features=10000, out_features=256, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

In [8]:
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [9]:
with torch.no_grad():
    outputs = model(X_test)
    preds = outputs.argmax(1)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("=== Test Accuracy ===")
print(accuracy_score(y_test, preds))

print("\n=== Classification Report ===")
print(classification_report(y_test, preds, target_names=["Not Interested", "Interested"]))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, preds))

=== Test Accuracy ===
0.8703703703703703

=== Classification Report ===
                precision    recall  f1-score   support

Not Interested       0.83      0.91      0.87       126
    Interested       0.92      0.83      0.87       144

      accuracy                           0.87       270
     macro avg       0.87      0.87      0.87       270
  weighted avg       0.87      0.87      0.87       270


=== Confusion Matrix ===
[[115  11]
 [ 24 120]]


In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score

metrics = {
    "Accuracy": accuracy_score(y_test, preds),
    "Precision": precision_score(y_test, preds),
    "Recall": recall_score(y_test, preds),
    "F1-score": f1_score(y_test, preds)
}

pd.DataFrame(metrics, index=["Score"])

,Accuracy,Precision,Recall,F1-score
Score,0.87037,0.916031,0.833333,0.872727
